In [1]:
from __future__ import division, print_function
import numpy as np
import pandas as pd
from sympy import *
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
from functools import partial, reduce
from itertools import combinations
from scipy.stats import percentileofscore
import affinity as af
import ratlib as rat

%matplotlib inline 

In [2]:
data = pd.read_excel('~/GIT/collinearity/database/sidis/expdata/1008.xlsx')
tab = pd.read_excel('~/GIT/collinearity/notebooks/SIDIS_Proximity/1008_Affinity.xlsx')

In [3]:
class Rfilter(object):

    def __init__(self, hadron='pi+', fudge=[0, 0]):
        #self.M = 0.938
        self.M = 0.94
        self.M2 = self.M**2
        self.set_Mh(hadron)
        

        self.fudge1 = fudge[0]
        self.fudge2 = fudge[1]

        self.MiT2 = 0.5 + 0.3 * fudge[0]
        self.MfT2 = 0.5 + 0.3 * fudge[1]
        self.kT = 0
    
        self.MX = 1.3
        self.Ma = 1.5
        self.Mb = 0.3
        self.deltaM = 0.3
        self.MJ = 0.3
        
        self.MiT = self.MiT2**0.5
        self.MfT = self.MfT2**0.5
        self.kT2 = self.kT**2
        
        # from paper, we need kf2, deltakT2, ki2
        # we need to have a way to vary them!
        self.kf       = 0
        self.ki       = 0
        self.kf2      = self.kf**2
        self.ki2      = self.ki**2
        self.deltakT  = 0
        self.deltakT2 = self.deltakT**2
        
    def set_ki(self,ki):
        self.ki = ki
        self.ki2 = ki**2
        
    def set_kf(self,kf):
        self.kf = kf
        self.kf2 = kf**2
        
    def set_kT(self,kT):
        self.kT = kT
        self.kT2 = kT**2
        
    def set_deltakT(self,deltakT):
        self.deltakT = deltakT
        self.deltakT2 = self.deltakT**2
        
    def set_Mh(self, hadron):
        if hadron == 'pi+':
            self.Mh = 0.135  #  
        if hadron == 'pi-':
            self.Mh = 0.135  #  
        if hadron == 'pi0':
            self.Mh = 0.135  #  
        if hadron == 'k+':
            self.Mh = 0.493
        if hadron == 'k-':
            self.Mh = 0.493
        if hadron == 'k+':
            self.Mh = 0.493
        if hadron == 'h+':
            #self.Mh = 0.135
            self.Mh = 0.14
        if hadron == 'h-':
            self.Mh = 0.135
        self.Mh2 = self.Mh**2
   
    def set_kibT(self,kibT):
        self.kibT = kibT
        self.kibT2 = kibT**2
        
    def set_phi(self,phi):
        self.phi = phi
    
    def set_xi(self,xi):
        self.xi = xi
        
    def set_zeta(self,zeta):
        self.zeta = zeta
    
    def set_MBT(self,PhT):
        self.MBT = np.sqrt( self.Mh2 + (PhT**2 ))
        
    def set_xn(self, x, Q2):
        self.xn = (2 * x / (1 + np.sqrt(1 + 4 * x**2 * self.M2 / Q2)))
    
    def set_zn(self, x, z, Q2, PhT, hadron):
        self.set_xn(x,Q2)
        self.set_Mh(hadron)
        self.set_MBT(PhT)
        self.zn =  self.xn * z/ (2 * x) * ( 1. + np.sqrt( 1.- 4. * self.M2 * self.MBT**2 * x**2 /  ( Q2**2 * z**2) ) )
        
        self.xn * z/ (2 * x) * ( 1. + np.sqrt( 1.- 4. * self.M2 * self.MBT**2 * x**2 /  ( Q2**2 * z**2) ) )
        
    def set_MfT(self):
        self.MfT = np.sqrt(self.kT**2 + self.MJ**2 +
                           self.deltaM**2 * self.fudge2)
    def set_MiT(self,x,Q2):
        self.set_xn(x, Q2)
        self.MiT = np.sqrt((self.xn * self.kT**2 + self.xn * (self.Ma + self.Mb / np.sqrt(self.xn))
                            ** 2 - (1 - self.xn) * self.xn * self.M2 + self.deltaM**2 * self.fudge1) / (1 - self.xn))
    def set_W2(self,x,Q2):
        self.W2 = Q2 * (1. - x) / x + self.M2
    
    def set_yh(self, x, z, Q2, PhT, hadron, sign=-1):
        self.set_xn(x, Q2)
        self.set_Mh(hadron)
        expy = Q2**0.5 * z * (Q2 - self.xn**2 * self.M2)\
            / (2 * self.M2 * self.xn**2 * (self.Mh2 + PhT**2)**0.5)\
            + sign * Q2**0.5 / (self.xn * self.M) * (z**2 * (Q2 - self.xn**2 * self.M2)**2
                                        / (4 * self.M2 * self.xn**2 * (self.Mh2 + PhT**2)) - 1)**0.5
        self.yh = np.log(expy)
        
    def get_ki2(self):
        return self.ki2
    
    def get_kf(self):
        return self.kf
    
    def get_W2(self):
        return self.W2

    def get_MBT(self):
        return self.MBT
        
    def get_MiT(self):
        return self.MiT

    def get_MfT(self):
        return self.MfT

    def get_xn(self):
        return self.xn

    def get_yh(self):
        return self.yh

    def get_zn(self):
        return self.zn
        
# rapidity of the target

    def get_yp(self, x, Q2):
        self.set_xn(x, Q2)
        return np.log(np.sqrt(Q2) / (self.xn * self.M))

    def get_yi(self, Q2):
        return 0.5 * np.log(Q2 / self.MiT**2)

    def get_yf(self, Q2):
        return -0.5 * np.log(Q2 / self.MfT**2)

    def get_MhT(self, PhT):
        return np.sqrt(self.Mh2 + PhT**2)
    
    def get_kibT(self,kibT):
        return self.kibT
    
    def get_M(self):
        return self.M
    
    def get_Mh(self):
        return self.Mh
    
    def get_Mh2(self):
        return self.Mh2
    
    def get_R(self, x, z, Q2, PhT, hadron):
        
        self.set_zn(x, z, Q2, PhT, hadron)
        zn2 = self.zn**2
        self.set_xn(x, Q2)
        znhat = self.zn/self.zeta
        xnhat = self.xn/self.xi
        qT = -PhT/self.zn
        
        # from paper..
        Ph_kf = ((((self.Mh2+zn2*qT**2)*znhat)/(2*self.zn))+(self.zn*((-znhat*qT+self.deltakT)**2+ \
                    self.kf2)/(2*znhat))) + self.zn*qT*(-znhat*qT+self.deltakT)
        Ph_ki = ((((self.Mh2+zn2*qT**2)*(xnhat*(self.ki2 + self.kibT2)))/(2*Q2*self.zn)) + \
                 (self.zn*Q2)/(2*xnhat))+self.zn*qT*self.kibT
        
        return np.abs(Ph_kf / Ph_ki)
    
    def get_R1S(self, x, z, Q2, PhT, hadron):
        
        self.set_zn(x, z, Q2, PhT, hadron)
        zn2 = self.zn**2
        self.set_xn(x, Q2)
        znhat = self.zn/self.zeta
        xnhat = self.xn/self.xi
        qT = -PhT/self.zn
        
        R1 = 1/self.zeta*(self.Mh2+self.zeta**2*(self.deltakT2+self.kf2))/((self.Mh2+zn2*qT**2)* \
             (self.ki2+self.kibT**2)*(self.xn/(self.zn*Q2*self.xi))+ (self.zn*Q2*self.xi/self.xn) + \
             2*self.zn*qT*self.kibT*np.cos(self.phi))
        return R1
        
        
# R0, Eq. (4.14)...
    def get_R0(self, Q2):
        """
        Collinearity ratio defined in the paper Eq. (4.15)
        """
        R0 = (np.maximum(np.maximum(self.ki**2/Q2,self.kf**2/Q2),self.kT**2/Q2))
        #print(R0)
        return  R0
        
        
# We call R1 in the new paper what was R in the previous...
    def get_R1(self, x, z, Q2, PhT, hadron):
        Rval.set_Mh(hadron)
        """
        Collinearity ratio defined in the paper Eq. (4.15)
        """
        return  self.get_R(x, z, Q2, PhT, hadron)

# R2 from Eq. (4.17)        
    def get_R2(self, x, z, Q2, PhT, hadron):
        self.set_Mh(hadron)
        self.set_zn(x, z, Q2, PhT, hadron)
        znhat = self.zn/self.zeta
        qT = -PhT/self.zn
        # from paper..
        return(np.abs( -(1.-znhat) - znhat*qT**2/Q2 - (1.-znhat)*self.kf2/(znhat*Q2) - \
                      self.deltakT2/(znhat*Q2) + 2.*qT*self.deltakT/Q2 ))
    
   


In [4]:
%%time

hadron = 'h+'
fudge = [0,0]
Rval = Rfilter(hadron = hadron,fudge = fudge)
Rval.set_kT(.1)
Rval.set_ki(.1)
Rval.set_kf(.1)
Rval.set_deltakT(.3)
Rval.set_kibT(0.2)
Rval.set_xi(0.3)
Rval.set_zeta(0.3)
Rval.set_phi(0)

x = .2
Q2 = 3**2
z = .3
pT = .1
M = 0.94
M_h = .14

Rval.set_xn(x, Q2)
xn = Rval.get_xn()
Rval.set_zn(x, z, Q2, pT, hadron)
zn = Rval.get_zn()
R1 = Rval.get_R1(x, z, Q2, pT, 'h+')
R2 = Rval.get_R2(x, z, Q2, pT, 'h+')
R1S2 = Rval.get_R1S( x, z, Q2, pT, hadron)
print('Rfilter     ',R1,'',R2)
print('Rfilter R1S ',R1S2)

# From ratios.py
M= .94
M_h=.14
x_bj = .2
Q = 3
z_h = .3
P_t = 0.1

    
xi = .3
zeta = .3
delta_k_t = .3 
k_i_t = .2 
M_ki = .1 
M_kf = .1 
szn = rat.zn(z_h,x_bj,Q,M,M_h,P_t)
q_t = -P_t/szn
eta = .1

R1S = rat.get_R1( M,M_h,x_bj,z_h,eta,Q,q_t,xi,zeta,delta_k_t,k_i_t,M_ki,M_kf)
R2S = rat.get_R2(M,M_h,x_bj,z_h,eta,Q,q_t,xi,zeta,delta_k_t,k_i_t,M_ki,M_kf)

sxN = rat.get_xN(M,M_h,x_bj,z_h,eta,Q,P_t,xi,zeta,delta_k_t,k_i_t,M_ki,M_kf)
sxn = rat.xnac(x_bj,M,Q)
print('ratlib.py zn = ',szn,' Rfilter zn = ',zn)
print('ratlib.py xn = ',sxn,'ratlib.py xN = ',sxN,' Rfilter xn = ',xn)
print('ratios.py   ',R1S,'',R2S)
print(0.023776044983519918,
0.04879270850634498)

Rfilter      0.023775178126167786  0.048792708506345
Rfilter R1S  0.02377517812616779
ratlib.py zn =  0.29878813895610384  Rfilter zn =  0.29878813895610384
ratlib.py xn =  0.19922068675196355 ratlib.py xN =  0.19922068675196355  Rfilter xn =  0.19922068675196355
ratios.py    0.023776044983519918  0.04879270850634498
0.023776044983519918 0.04879270850634498
CPU times: user 400 µs, sys: 3.86 ms, total: 4.26 ms
Wall time: 3.42 ms


In [5]:
# Proximity is defined for a sphere of a selected radius(p) by the following function
# This notebook does not use this function to compare ratios.py with Rfilter

def Proximity(data,domain1,domain2,domainPhi,p,N,fudge):
    """
    data    = any Pandas DataFrame should do as long as it contains x,z,Q2, and pT
    domain1 = 
    domain2 = 
    p       = radius of sphere representing the demarcation of the proximity
    hadron  = any hadron found in the data
    fudge   = idk what fudge is so I did the right thing and passed it along 
    """
    
    #data.loc[:,'proximity'] = 0.
    
    
    for i in range(len(data)):
        
        hadron = data.loc[i,'hadron']
        Rval = Rfilter(hadron = hadron,fudge = fudge)
        # Creates vectors of length N made from uniform random numbers in defined domains.
        zeta = np.random.uniform(data.loc[i,'z']-.1,data.loc[i,'z']+.1,N)
        ki = np.random.uniform(domain1[0],domain1[1],N)
        kf = np.random.uniform(domain1[0],domain1[1],N)
        kT = np.random.uniform(domain1[0],domain1[1],N)
        kibT = np.random.uniform(domain1[0],domain1[1],N)
        deltakT = np.random.uniform(domain2[0],domain2[1],N)
        xi = np.linspace(domain2[0],domain2[1],N)
        phi = np.linspace(domainPhi[0],domainPhi[1],N)
        
        # Sets our parameters for Rfilter
        Rval.set_ki(ki)
        Rval.set_kf(kf)  
        Rval.set_kT(kT)
        Rval.set_kibT(kibT)
        Rval.set_deltakT(deltakT)
        Rval.set_zeta(zeta)
        Rval.set_phi(phi)
        Rval.set_xi(xi)
        
        # Gets Ratios from Rfilter 
        R0 = Rval.get_R0(data.loc[i,'Q2'])             
        R1 = Rval.get_R1(data.loc[i,'x'], data.loc[i,'z'], data.loc[i,'Q2'], data.loc[i,'pT'], hadron)
        R2 = Rval.get_R2(data.loc[i,'x'], data.loc[i,'z'], data.loc[i,'Q2'], data.loc[i,'pT'], hadron)
        SR1 = Rval.get_R1S(data.loc[i,'x'], data.loc[i,'z'], data.loc[i,'Q2'], data.loc[i,'pT'], hadron)
        
        # proximity uses Rfiler for R0, R1, R2 with Alexei's method of proximity using error to define what
        # is inside sphere of radius p(typically p = 1)
        data.loc[i,'proximity'] = sum(1 for i in range(N) if (R0[i]**2+R1[i]**2+R2[i]**2)<p)/N
        
        # affinity uses Rfilter for R1 and R2 but with Sterlings approach for Saffinity %
        data.loc[i,'affinity']  = sum([1 for i in range(N) if SR1[i]<0.3 if R2[i]<0.3])/N  
        
        # Input for Sterlings code
        x=data.loc[i,'x']
        z=data.loc[i,'z']
        Q=data.loc[i,'Q2']**.5
        val=data.loc[i,'value']
        P_t=data.loc[i,'pT']
        
        #M= Rval.get_M()
        #M_h = Rval.get_Mh()
        M= .94
        M_h = .14
        z_N = rat.zn(z,x,Q,M,M_h,P_t)
        q_t = -P_t/z_N
        
        #M_h=.14
        eta = rat.get_eta( M,M_h,x,z,Q,q_t)
        
        sR1 = rat.get_R1( M,M_h,x_bj,z_h,eta,Q,q_t,xi,zeta,deltakT,kibT,ki,kf)
        sR2 = rat.get_R2( M,M_h,x_bj,z_h,eta,Q,q_t,xi,zeta,deltakT,kibT,ki,kf)
        
        # Saffinity uses Sterlings equations in ratlib.py
        data.loc[i,'Saffinity'] = sum([1 for i in range(N) if sR1[i]<0.3 if sR2[i]<0.3])/N 
        if i == 2:
            for j in range(len(R1)):
                print('R1 Rfilter =          ',R1[j],'\nR1 Sterling Rfilter = ',SR1[j],'\nratlib R1 =           ',sR1[j],'\n')
    return data

In [6]:
%%time
N = 1000
p = 1
fudge = [0,0]
domain1 = [0,.1]
domain2 = [0.3,0.4]
domainPhi = [0,0]
data = Proximity(data,domain1,domain2,domainPhi,p,N,fudge)

R1 Rfilter =           0.017426752388386108 
R1 Sterling Rfilter =  0.017426752388386108 
ratlib R1 =            0.014832315058521216 

R1 Rfilter =           0.018800930475390568 
R1 Sterling Rfilter =  0.018800930475390595 
ratlib R1 =            0.01598753122451967 

R1 Rfilter =           0.02662564903761558 
R1 Sterling Rfilter =  0.026625649037615617 
ratlib R1 =            0.022627872553268494 

R1 Rfilter =           0.015276184660009295 
R1 Sterling Rfilter =  0.0152761846600093 
ratlib R1 =            0.012991232888875062 

R1 Rfilter =           0.013504065728870175 
R1 Sterling Rfilter =  0.013504065728870173 
ratlib R1 =            0.011469821026971512 

R1 Rfilter =           0.016008635646725316 
R1 Sterling Rfilter =  0.016008635646725305 
ratlib R1 =            0.013622843933102447 

R1 Rfilter =           0.014048659872078805 
R1 Sterling Rfilter =  0.014048659872078794 
ratlib R1 =            0.011945844106362445 

R1 Rfilter =           0.015198999001667438 
R1 Ster

ratlib R1 =            0.01609881542523577 

R1 Rfilter =           0.017096021348909998 
R1 Sterling Rfilter =  0.01709602134891001 
ratlib R1 =            0.014537365395794815 

R1 Rfilter =           0.014525793185419439 
R1 Sterling Rfilter =  0.014525793185419461 
ratlib R1 =            0.012364317375844949 

R1 Rfilter =           0.01225714203800803 
R1 Sterling Rfilter =  0.01225714203800801 
ratlib R1 =            0.010413435068696332 

R1 Rfilter =           0.02042636272002059 
R1 Sterling Rfilter =  0.020426362720020574 
ratlib R1 =            0.017357226766540362 

R1 Rfilter =           0.012286552878298674 
R1 Sterling Rfilter =  0.012286552878298667 
ratlib R1 =            0.010453316932468454 

R1 Rfilter =           0.017299951159309233 
R1 Sterling Rfilter =  0.017299951159309226 
ratlib R1 =            0.01472092796930759 

R1 Rfilter =           0.013946243834532971 
R1 Sterling Rfilter =  0.013946243834532942 
ratlib R1 =            0.011854544201887692 

R1 Rfilt

ratlib R1 =            0.01037225443748796 

R1 Rfilter =           0.013000103697536193 
R1 Sterling Rfilter =  0.01300010369753617 
ratlib R1 =            0.01104318248136736 

R1 Rfilter =           0.020305129607491333 
R1 Sterling Rfilter =  0.020305129607491312 
ratlib R1 =            0.017265996226559704 

R1 Rfilter =           0.01192192089969349 
R1 Sterling Rfilter =  0.011921920899693473 
ratlib R1 =            0.010135184742443532 

R1 Rfilter =           0.016794426500795327 
R1 Sterling Rfilter =  0.016794426500795268 
ratlib R1 =            0.014271027409903063 

R1 Rfilter =           0.012970795513525249 
R1 Sterling Rfilter =  0.012970795513525245 
ratlib R1 =            0.011033869108693606 

R1 Rfilter =           0.01892123607583833 
R1 Sterling Rfilter =  0.018921236075838302 
ratlib R1 =            0.01607506974619093 

R1 Rfilter =           0.017391096502838155 
R1 Sterling Rfilter =  0.017391096502838134 
ratlib R1 =            0.0147749001672671 

R1 Rfilter

R1 Rfilter =           0.01722927172807545 
R1 Sterling Rfilter =  0.017229271728075504 
ratlib R1 =            0.014662163051600341 

R1 Rfilter =           0.012136307895853826 
R1 Sterling Rfilter =  0.012136307895853819 
ratlib R1 =            0.010323840780882311 

R1 Rfilter =           0.014120863107371203 
R1 Sterling Rfilter =  0.014120863107371187 
ratlib R1 =            0.011997209942774943 

R1 Rfilter =           0.013733933791140839 
R1 Sterling Rfilter =  0.013733933791140813 
ratlib R1 =            0.011685690325461497 

R1 Rfilter =           0.0116938598801768 
R1 Sterling Rfilter =  0.011693859880176787 
ratlib R1 =            0.009935210665268429 

R1 Rfilter =           0.012224444282016564 
R1 Sterling Rfilter =  0.012224444282016576 
ratlib R1 =            0.010389022135078458 

R1 Rfilter =           0.0156386441023986 
R1 Sterling Rfilter =  0.015638644102398597 
ratlib R1 =            0.01329475917501882 

R1 Rfilter =           0.010766728913529899 
R1 Sterli

In [22]:
# Trying out symbolic math operations in python using sympy

R1_str_expr_denominator = '1/2*((M_B**2+z_N**2*q_T**2)*(k_f**2+k_iT**2)*x_N/(z_N*Q**2*xi)+z_N*Q**2*xi/x_N+2*z_N*q_T*k_iT*cos(phi_i))'
denominator = sympify(R1_str_expr_denominator)
R1_str_expr_numerator = "1/(2*zeTa)*(M_B**2+zeTa**2*(delta_kT**2+k_f**2))"
numerator = sympify(R1_str_expr_numerator)
R1 = sympify(numerator/denominator)
R1

(M_B**2 + zeTa**2*(delta_kT**2 + k_f**2))/(2*zeTa*(Q**2*xi*z_N/(2*x_N) + k_iT*q_T*z_N*cos(phi_i) + x_N*(M_B**2 + q_T**2*z_N**2)*(k_f**2 + k_iT**2)/(2*Q**2*xi*z_N)))

In [23]:
R1 = R1.subs('q_T','-p_T/z_N')
R1 = R1.subs('z_N','x_N*z/(2*x)*(1+sqrt(1-(4*M**2*M_BT**2*x**2)/(Q**4*z**2)))')
R1 = R1.subs('x_N','2*x/(1+sqrt(1+4*x**2*M**2/Q**2))')
R1 = R1.subs('M_BT','sqrt(M_B**2+p_T**2)')
simplify(R1)

2*Q**2*x*xi*z*(M_B**2 + zeTa**2*(delta_kT**2 + k_f**2))*(sqrt((-4*M**2*x**2*(M_B**2 + p_T**2) + Q**4*z**2)/(Q**4*z**2)) + 1)/(zeTa*(Q**4*xi**2*z**2*(sqrt((-4*M**2*x**2*(M_B**2 + p_T**2) + Q**4*z**2)/(Q**4*z**2)) + 1)**2 - 4*Q**2*k_iT*p_T*x*xi*z*(sqrt((-4*M**2*x**2*(M_B**2 + p_T**2) + Q**4*z**2)/(Q**4*z**2)) + 1)*cos(phi_i) + 4*x**2*(M_B**2 + p_T**2)*(k_f**2 + k_iT**2)))

In [24]:
# Checking R1 for ratios.py parameter values

M= .94
M_B=.14
x = .2
Q = 3
z = .3
p_T = 0.1 
xi = .3
zeTa = .3
delta_kT = .3 
k_iT = .2 
M_ki = .1 
M_kf = .1 
eta = .1

R1 = R1.subs({'Q':3,'z':.3,'xi':.3,'zeTa':.3,'delta_kT':.3,'k_iT':.2,'x':.2,'M':.94,'M_B':.14,'p_T':.1, \
              'k_f':.1,'k_i':.1,'phi_i':0})

In [27]:
print(R1)

0.0237751781261678
